# Tutorial 1: Hello INR - Your First Neural Field

**Time:** 15 minutes  
**Goal:** Understand what INRs are by fitting one to a simple image

---

## What is an Implicit Neural Representation?

Traditional way to store an image:
```python
image = np.array([[r,g,b], [r,g,b], ...])  # A grid of pixels
```

INR way:
```python
def image(x, y):
    return neural_network([x, y])  # A function!
```

**Key insight:** Instead of storing pixel values, we store a *function* that generates them.

Let's see this in action! 🚀

In [ ]:
# Imports
import sys
sys.path.append('..')

import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from inr_toolkit.models import SIREN
from inr_toolkit.training import Trainer
from inr_toolkit.utils import get_image_coordinates, load_image, plot_comparison, psnr

# Use GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

## Step 1: Create a Simple Test Image

Let's create a simple gradient image to fit.

In [ ]:
# Create a simple gradient image
height, width = 128, 128
x = np.linspace(0, 1, width)
y = np.linspace(0, 1, height)
X, Y = np.meshgrid(x, y)

# Create RGB gradient
image = np.stack([
    X,           # Red increases left to right
    Y,           # Green increases top to bottom  
    1 - X * Y,   # Blue decreases
], axis=-1)

plt.figure(figsize=(6, 6))
plt.imshow(image)
plt.title('Our Test Image (128x128)')
plt.axis('off')
plt.show()

print(f'Image shape: {image.shape}')

## Step 2: Prepare Training Data

For an INR, our training data is:
- **Input:** (x, y) coordinates
- **Target:** (r, g, b) color values

We're teaching the network: "When I give you coordinate (x,y), output color (r,g,b)"

In [ ]:
# Get coordinates for every pixel
coords = get_image_coordinates(height, width)  # Shape: (128*128, 2)
colors = torch.from_numpy(image.reshape(-1, 3).astype(np.float32))  # Shape: (128*128, 3)

print(f'Coordinates shape: {coords.shape}')
print(f'Colors shape: {colors.shape}')
print(f'\nExample:')
print(f'  Coordinate: {coords[0]}')
print(f'  Color: {colors[0]}')

## Step 3: Create the Neural Field

We'll use SIREN - a network with sine activations that works well for smooth images.

In [ ]:
# Create SIREN model
model = SIREN(
    in_dim=2,        # Input: (x, y)
    out_dim=3,       # Output: (r, g, b)
    hidden_dim=256,  # Width of hidden layers
    num_layers=4,    # Depth of network
)

print(f'Model has {model.count_parameters():,} parameters')
print(f'\nModel architecture:')
print(model)

## Step 4: Train the Neural Field

Now the magic happens - we optimize the network to memorize the image!

In [ ]:
# Create trainer
trainer = Trainer(model, lr=1e-4, device=device)

# Train!
trainer.fit(coords, colors, epochs=500, log_every=100)

## Step 5: Visualize Results

Let's see how well the network learned to represent our image!

In [ ]:
# Generate reconstruction
model.eval()
with torch.no_grad():
    coords_tensor = coords.to(device)
    reconstruction = model(coords_tensor).cpu().numpy()
    reconstruction = reconstruction.reshape(height, width, 3)

# Calculate PSNR
psnr_value = psnr(
    torch.from_numpy(reconstruction),
    torch.from_numpy(image)
)

# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(image)
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(np.clip(reconstruction, 0, 1))
axes[1].set_title(f'Reconstruction\nPSNR: {psnr_value:.2f} dB')
axes[1].axis('off')

difference = np.abs(image - reconstruction)
axes[2].imshow(difference)
axes[2].set_title('Absolute Difference\n(Amplified for visibility)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print(f'\nReconstruction quality: {psnr_value:.2f} dB PSNR')
print(f'(Higher is better, >30 dB is good)')

## Step 6: The Cool Part - Resolution Independence!

Since our "image" is now a function, we can query it at ANY resolution! 🤯

In [ ]:
# Render at 2x resolution (256x256)
high_res_coords = get_image_coordinates(256, 256).to(device)

with torch.no_grad():
    high_res = model(high_res_coords).cpu().numpy()
    high_res = high_res.reshape(256, 256, 3)

plt.figure(figsize=(8, 8))
plt.imshow(np.clip(high_res, 0, 1))
plt.title('Same Network, 2x Resolution (256x256)!')
plt.axis('off')
plt.show()

print('We trained on 128x128, but can render at any resolution!')
print('The network learned the underlying FUNCTION, not just pixel values.')

## Summary

**What you learned:**
1. ✅ INRs represent data as continuous functions, not discrete grids
2. ✅ Training data is (coordinates → values) pairs
3. ✅ After training, you can query at ANY resolution
4. ✅ SIREN uses sine activations to fit smooth signals

**Key insight:** 
```
Traditional: Store N×M pixels
INR: Store a function (neural network weights)
```

---

## Try it yourself!

Experiment with:
- Different `hidden_dim` (try 128, 512)
- Different `num_layers` (try 2, 6)
- More training epochs
- Your own images!

**Next:** [Tutorial 2 - Fourier Features](02_fourier_features.ipynb) to learn why this is hard for standard MLPs!